docker run -d --name sparkbook -p 8881:8888 -v "$PWD":/home/jovyan/work jupyter/pyspark-notebook start.sh jupyter lab --LabApp.token=''

m5.8xlarge  128GiB  
r5.4xlarge  128GiB  
x1e.xlarge  122GiB  
x1.16xlarge 976GiB  
i3.4xlarge  122GiB  
m5n.4xlarge	64 GiB UT25Gbps  

In [ ]:
import requests
from bs4 import BeautifulSoup
import pyspark as ps
import numpy as np
import time

response_cache = {}

def http_get(url):
    if response := response_cache.get(url):
        return response
    else:
        response = requests.request(url=url, method="GET")
        counties_list_html = response.content
        response_cache[url] = counties_list_html
        
        return response_cache[url]

spark = (ps.sql.SparkSession
         .builder
         .master('local[8]')
         .appName('lecture')
         .getOrCreate())

sc = spark.sparkContext

In [ ]:
Holmes_County,_Ohio

In [ ]:
counties_list_url = "https://en.wikipedia.org/wiki/List_of_United_States_counties_and_county_equivalents"

def wikipedia_export_get(title):
    xml = http_get("https://en.wikipedia.org/wiki/Special:Export/" + title)
    
    if b'<redirect title' in xml:
        return wikipedia_export_get(title=BeautifulSoup(xml).select('redirect')[0]['title'])
    else:
        return xml

def wikipedia_standard_url(title):
    return "https://en.wikipedia.org/wiki/" + title

# def wikipedia_standard_url_encode(title):
#     import urllib.parse
#     return "https://en.wikipedia.org/wiki/" + urllib.parse.quote(title)

def wikipedia_counties_titles():
    soup = BeautifulSoup(http_get(counties_list_url))
    print(soup.select('.wikitable.sortable caption big'))
    
    rows = soup.select('.wikitable.sortable tbody tr')
    anchors = sum([row.select('td a')[:1] for row in rows], [])

    urls = [a['href'][len('/wiki/'):] for a in anchors]
    
    return urls
    
def wikipedia_communities_subheadings(text):
    import re

    regex = r"(^==\s?Communities\s?==)(.+?)(^==[^=].+?[^=]==)"
    post_communities = re.findall(regex, text, re.MULTILINE + re.DOTALL)
    
    if len(post_communities) == 0:
        return []

    # print(post_communities[0])
    post_communities = post_communities[0]
    
    regex = r"===(.+)==="
    res = [result.strip() for result in re.findall(regex, post_communities[1])]

    return res

time1 = time.time()
wikipedia_counties_titles()
f'time.{time.time() - time1}'

In [ ]:
def wikipedia_communities_extract(text):
    import re

    regex = r"(^==\s?Communities\s?==)(.+?)(^==[^=].+?[^=]==)"
    post_communities = re.findall(regex, text, re.MULTILINE + re.DOTALL)
    
    regex = r"(^==\s?Municipalities\s?==)(.+?)(^==[^=].+?[^=]==)"
    municipalities = re.findall(regex, text, re.MULTILINE + re.DOTALL)
    
    regex = r"(^==\s?Cities and communities\s?==)(.+?)(^==[^=].+?[^=]==)"
    cities_and_communities = re.findall(regex, text, re.MULTILINE + re.DOTALL)
    
    regex = r"(^==\s?Community\s?==)(.+?)(^==[^=].+?[^=]==)"
    community = re.findall(regex, text, re.MULTILINE + re.DOTALL)
    
    types = []
    if len(post_communities) > 0:
        types.append("Communities")
        
    if len(municipalities) > 0:
        types.append("Municipalities")
        if len(municipalities) > len(post_communities):
            post_communities = municipalities

    if len(cities_and_communities) > 0:
        types.append("Cities and communities")
        if len(cities_and_communities) > len(post_communities):
            post_communities = cities_and_communities

    if len(community) > 0:
        types.append("Community")
        if len(community) > len(post_communities):
            post_communities = community
    
    if len(post_communities) == 0:
        return []

    post_communities = post_communities[0]
    
    regex = r"===(.+)==="
    communities_subheadings = [result.strip() for result in re.findall(regex, post_communities[1])]
    
    ## --
    
    # print(re.findall(r"\*\s*\[", post_communities[1]))
    regex = r"\*\s*\[\[(.+?),(.+?)\|(.+?)\]\]"
    communities = re.findall(regex, post_communities[1])
    
    regex = r"\*\s*\[\[(.+?)\|(.+?)\]\]"
    communities_all = re.findall(regex, post_communities[1])
    
    regex = r"\*\s*\[\[([^|]+?)\]\]"
    communities_untitled = re.findall(regex, post_communities[1])
        
    a = set([c[2] for c in communities])
    b = set([c[1] for c in communities_all])
    difference = b.difference(a).union(a.difference(b))
    
    return communities_subheadings, [c[0] for c in communities_all], difference, communities_untitled, types

In [ ]:
title = 'Garvin_County,_Oklahoma'
print(wikipedia_standard_url(title))
text = BeautifulSoup(wikipedia_export_get(title)).select('text')[0].getText()
# print(text)
wikipedia_communities_extract(text)

---

### Init!

In [ ]:
wikipedia_export_get(twenty[15])

In [ ]:
%%time

twenty = wikipedia_counties_titles()

counties_export_text = (sc.parallelize(twenty)
                          .map(wikipedia_export_get)
                          .map(lambda export: BeautifulSoup(export).select('text')[0].getText())
                          .cache())

In [ ]:
communities_all = (counties_export_text.map(wikipedia_communities_extract)
                                       .filter(lambda x: x != [])
                                       .map(lambda x: x[1]))

# out = [o for o in out if o[0] == tuple() and ',_Indiana' in o[1]]
# out = [o for o in out if o[0] != tuple() and ',_Puerto_Rico' not in o[1]]
# out = [o for o in out if o[0] != tuple() and ',_Indiana' in o[1]]
# out = [o for o in out if o[0] != tuple() and ',_Virginia' in o[1]]
# out = [o for o in out if o[0] != tuple() and ',_Alaska' in o[1]]

communities_all_1 = list(communities_all.map(set).reduce(set.union))

In [ ]:
len(communities_all_1)

In [ ]:
with open('communities_all.txt', 'w') as f:
    f.write("\n".join(communities_all_1))

In [ ]:
from IPython.display import HTML
HTML('<br>'.join([f'<a href="{wikipedia_standard_url(title)}">{title}</a>' for title in np.random.choice(communities_all_1, 250)]))

---
#### Newtypes

---

#### Investigations

In [ ]:
print(BeautifulSoup(wikipedia_export_get(np.random.choice(communities_all_1))))